In [ ]:
!nvidia-smi

Thu Apr 21 19:19:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install jupyter_http_over_ws


In [ ]:
#!jupyter serverextension enable --py jupyter_http_over_ws

In [ ]:
!pip install transformers==4.16.2

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 6.6 MB 37.6 MB/s 
     |████████████████████████████████| 895 kB 36.3 MB/s 
     |████████████████████████████████| 596 kB 38.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 582 kB 5.3 MB/s 
     |████████████████████████████████| 408 kB 62.7 MB/s 
     |████████████████████████████████| 136 kB 52.7 MB/s 
     |████████████████████████████████| 1.1 MB 40.0 MB/s 
     |████████████████████████████████| 144 kB 40.5 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 271 kB 39.4 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertTokenizer,
    BertModel,
    AdamW,
    get_linear_schedule_with_warmup,
    AutoConfig,
    AutoTokenizer,
    TFAutoModelForSequenceClassification
)
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import sklearn
import pytorch_lightning as pl
import torch
from torchmetrics.functional.classification import auroc

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train_set = pd.read_csv('gdrive/MyDrive/BERT/Datasets/train.csv', index_col=0)
del train_set["created_at"]
del train_set["id"]

dev_set = pd.read_csv('gdrive/MyDrive/BERT/Datasets/dev.csv', index_col=0)
del dev_set["created_at"]
del dev_set["id"]

test_set = pd.read_csv('gdrive/MyDrive/BERT/Datasets/test.csv', index_col=0)
del test_set["created_at"]
del test_set["id"]

In [ ]:
train_set.shape, dev_set.shape

((1115, 2), (132, 2))

In [ ]:
model_name = 'digitalepidemiologylab/covid-twitter-bert'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/421 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
train_list = train_set.values.tolist()
dev_list = dev_set.values.tolist()
test_list = train_set.values.tolist()

train_text = train_set['text'].values.tolist()
dev_text = dev_set['text'].values.tolist()
test_text = test_set['text'].values.tolist()

In [ ]:
train_x = tokenizer(train_text,
                    add_special_tokens=True,
                    max_length=512,
                    return_token_type_ids=False,
                    padding="max_length",
                    return_attention_mask=True ,
                    truncation=True,
                    return_tensors="pt"
                   )



In [ ]:
train_x.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
train_x['input_ids'].shape, train_x["attention_mask"].shape

(torch.Size([1115, 512]), torch.Size([1115, 512]))

In [ ]:
train_x["input_ids"][15].squeeze().shape

torch.Size([512])

In [ ]:
print(tokenizer.convert_ids_to_tokens(train_x["input_ids"][15].squeeze()))

['[CLS]', 'so', 'this', 'co', '##vid', '-', '19', 'vaccine', 'is', 'still', 'in', 'phase', '3', 'of', 'testing', '.', '.', '.', '[UNK]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [ ]:
LABEL_COLUMNS = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]

In [ ]:
class CoVaxLiesDataset(Dataset):
  def __init__(self, data: pd.DataFrame, tokenizer: AutoTokenizer, max_token_len: int=128):
    self.tokenizer = tokenizer
    self.data=data
    self.max_token_len = max_token_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    comment_text = data_row["text"]
    temp = ast.literal_eval(data_row['misinfo'])
    #labellist = {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0}
    #labellist = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    labellist = [int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0)]
    keys = list(temp.keys())
    for key in keys:
      labelbool = None
      labelstr = temp[key]
      if labelstr == 'not_relevant':
        labelbool = int(0)
      if labelstr == 'agree':
        labelbool = int(1)
      if labelstr == 'disagree':
        labelbool = int(1)
      if labelstr == 'no_stance':
        labelbool = int(1)
      labellist[int(key) - 1] = int(labelbool)


    #encoder = preprocessing.LabelEncoder()

    encoding = self.tokenizer.encode_plus(
        comment_text,
        add_special_tokens=True,
        max_length=self.max_token_len,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt"
    )

    return dict(
        comment_text=comment_text,
        input_ids=encoding["input_ids"].flatten(),
        attention_mask=encoding["attention_mask"].flatten(),
        labels=torch.FloatTensor(labellist)
    )

In [ ]:
train_dataset = CoVaxLiesDataset(train_set, tokenizer)

In [ ]:
sample_item = train_dataset[56]

In [ ]:
sample_item.keys()

dict_keys(['comment_text', 'input_ids', 'attention_mask', 'labels'])

In [ ]:
sample_item["comment_text"]

"@vixenin_vegas's account is temporarily unavailable because it violates the Twitter Media Policy. Learn more."

In [ ]:
sample_item["labels"]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0.])

In [ ]:
bert_model = BertModel.from_pretrained(model_name, return_dict=True)

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
prediction = bert_model(sample_item["input_ids"].unsqueeze(dim=0), sample_item["attention_mask"].unsqueeze(dim=0))

In [ ]:
prediction.last_hidden_state.shape, prediction.pooler_output.shape

(torch.Size([1, 128, 1024]), torch.Size([1, 1024]))

In [ ]:
class CoVaxLiesDataModule(pl.LightningDataModule):
  def __init__(self, train_df, test_df, dev_df, tokenizer, batch_size=8, max_token_len=128):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.dev_df = dev_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.max_token_len = max_token_len

  def setup(self):
    self.train_dataset = CoVaxLiesDataset(
        self.train_df,
        self.tokenizer,
        self.max_token_len
    )

    self.test_dataset = CoVaxLiesDataset(
        self.test_df,
        self.tokenizer,
        self.max_token_len
    )

    self.dev_dataset = CoVaxLiesDataset(
        self.dev_df,
        self.tokenizer,
        self.max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=4
    )

  def dev_dataloader(self):
    return DataLoader(
        self.dev_dataset,
        batch_size=self.batch_size,
        num_workers=4
    )

  def test_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        num_workers=4
    )


In [ ]:
N_EPOCHS = 8
BATCH_SIZE = 5


data_module = CoVaxLiesDataModule(train_set, test_set, dev_set, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()



train_dataloader = data_module.train_dataloader()
dev_dataloader = data_module.dev_dataloader()
test_dataloader = data_module.test_dataloader()




In [ ]:
for i, name in enumerate(LABEL_COLUMNS):
  print(i)
  print(name)

0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20


##Modeling
Evaluation


# New Section

In [ ]:
criterion = nn.BCELoss()



In [ ]:
class CoVaxLiesTagger(pl.LightningModule):
  def __init__(self, n_classes: int, steps_per_epoch=None, n_epochs=None):
    super().__init__()
    self.bert = BertModel.from_pretrained(model_name, return_dict=True)
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.steps_per_epoch = steps_per_epoch
    self.n_epochs = n_epochs
    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)
    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_size):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True, batch_size=batch_size)
    return {"loss": loss, "prediction": outputs, "labels": labels}

  def validation_step(self, batch, batch_size):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True, batch_size=batch_size)
    return loss

  def test_step(self, batch, batch_size):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True, batch_size=batch_size)
    return loss

  def training_epoch_end(self, outputs):

    labels = []
    predictions = []

    for output in outputs:
      for out_labels in output["labels"].detach().cpu():
        #labels.append(out_labels.argmax())
        temp = [int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0)]
        temp[out_labels.argmax()] = 1

        labels.append(temp)


      for out_predictions in output["prediction"].detach().cpu():
        #predictions.append(out_predictions.argmax())
        temp2 = [int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0),int(0)]
        temp2[out_predictions.argmax()] = 1

        predictions.append(temp2)



    labels = torch.tensor(labels)
    predictions = torch.tensor(predictions)

    print(labels)
    print(predictions)
    print("PREDICTIONS:")
    for i, name in enumerate(LABEL_COLUMNS):
      print(predictions[i])

    for i, name in enumerate(LABEL_COLUMNS):
      #print("PREDICTIONS:")
      #print(predictions[i])
      #print(predictions.shape)

      roc_score = auroc(predictions[:, np.newaxis], labels[:, np.newaxis], num_classes=1)
      self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", roc_score, self.current_epoch)


  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=1e-5)

    warmup_steps = self.steps_per_epoch // 3
    total_steps = self.steps_per_epoch = self.n_epochs - warmup_steps

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        warmup_steps,
        total_steps
    )

    return [optimizer], [scheduler]


In [ ]:
model = CoVaxLiesTagger(n_classes=20,
                        steps_per_epoch=len(train_set) // BATCH_SIZE,
                        n_epochs=N_EPOCHS
                        )

Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#prediction = bert_model(sample_item["input_ids"].unsqueeze(dim=0), sample_item["attention_mask"].unsqueeze(dim=0))

_, predictions = model(sample_item["input_ids"].unsqueeze(dim=0),
                       sample_item["attention_mask"].unsqueeze(dim=0),
                       )

In [ ]:
"""def get_predictions(data_set):
  y_true = []
  y_pred = []
  for i in range(len(data_set)):
    item = data_set[i]
    y_true.append(argmax(item["labels"]) + 1)
    _, predictions = model(item["input_ids"].unsqueeze(dim=0),
                       item["attention_mask"].unsqueeze(dim=0),
        )
    y_pred.append(argmax(predictions) + 1)
  score = sklearn.metrics.accuracy_score(y_true, y_pred)
  return score"""


'def get_predictions(data_set):\n  y_true = []\n  y_pred = []\n  for i in range(len(data_set)):\n    item = data_set[i]\n    y_true.append(argmax(item["labels"]) + 1)\n    _, predictions = model(item["input_ids"].unsqueeze(dim=0),\n                       item["attention_mask"].unsqueeze(dim=0),\n        )\n    y_pred.append(argmax(predictions) + 1)\n  score = sklearn.metrics.accuracy_score(y_true, y_pred)\n  return score'

In [ ]:
predictions

tensor([[0.5715, 0.5663, 0.6769, 0.4396, 0.5744, 0.4219, 0.4161, 0.4631, 0.2038,
         0.3755, 0.4547, 0.7748, 0.3705, 0.6345, 0.4966, 0.5223, 0.6850, 0.4221,
         0.3764, 0.5454]], grad_fn=<SigmoidBackward0>)

In [ ]:
trainer = pl.Trainer(max_epochs=N_EPOCHS, gpus=1, progress_bar_refresh_rate=30)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, train_dataloader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name       | Type      | Params
-----------------------------------------
0 | bert       | BertModel | 335 M 
1 | classifier | Linear    | 20.5 K
2 | criterion  | BCELoss   | 0     
-----------------------------------------
335 M     Trainable params
0         Non-trainable params
335 M     Total params
1,

Training: 0it [00:00, ?it/s]

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
PREDICTIONS:
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
model.freeze()

In [ ]:
test_comment = "@HeidiTurid Adverse reactions following the Pfizer-BioNTech COVID-19 Vaccine that have been reported in clinical trials include injection site pain, fatigue, headache, muscle pain, chills, joint pain, fever, injection site swelling, injection site redness, nausea https://t.co/BDCSUQfbZP"

In [ ]:
#test_set = dev_set["text"].tolist()
#test_set = CoVaxLiesDataset(dev_set, tokenizer)
#print(test_set[15])

trainer.validate(dataloaders=dev_dataloader)



Getting Predictions from Train, Dev, and Test datasets

In [ ]:
def get_predictions(data_set):
  y_true = []
  y_pred = []

  #print(item['labels'])
  # y_true = np.argmax(data_set["labels"]) + 1
  # _, predictions = model(data_set["input_ids"].unsqueeze(dim=0),
  #                        data_set["attention_mask"].unsqueeze(dim=0))

  for x in data_set:
    y_true.append(np.argmax(x["labels"]) + 1)
    _, predictions = model(x["input_ids"].unsqueeze(dim=0),
                           x["attention_mask"].unsqueeze(dim=0))
    pred = predictions.detach().numpy()
    y_pred.append(np.argmax(pred) + 1)
    print(np.argmax(pred) + 1)
    print(np.argmax(x["labels"]) + 1)


  score = sklearn.metrics.accuracy_score(y_true, y_pred)

  return score

In [ ]:
#train_dataset = CoVaxLiesDataset(train_set, tokenizer)
dev_dataset = CoVaxLiesDataset(dev_set, tokenizer)
test_dataset = CoVaxLiesDataset(test_set, tokenizer)

In [ ]:
print(dev_dataset[1])
print(dev_dataset)

In [ ]:
print(get_predictions(dev_dataset))

In [ ]:
print(get_predictions(train_dataset))

In [ ]:
print(get_predictions(test_dataset))

20
tensor(20)
10
tensor(11)
10
tensor(9)
4
tensor(1)
1
tensor(5)
3
tensor(20)
10
tensor(1)
10
tensor(2)
10
tensor(4)
10
tensor(8)
10
tensor(1)
10
tensor(1)
10
tensor(1)
4
tensor(1)
10
tensor(20)
10
tensor(1)
8
tensor(1)
8
tensor(9)
8
tensor(9)
8
tensor(10)
10
tensor(1)
8
tensor(7)
10
tensor(4)
8
tensor(10)
1
tensor(1)
10
tensor(11)
10
tensor(16)
10
tensor(1)
8
tensor(1)
10
tensor(1)
10
tensor(5)
10
tensor(1)
10
tensor(3)
10
tensor(1)
9
tensor(1)
8
tensor(7)
10
tensor(16)
8
tensor(8)
10
tensor(2)
10
tensor(4)
10
tensor(1)
10
tensor(1)
10
tensor(1)
8
tensor(9)
8
tensor(2)
8
tensor(1)
8
tensor(9)
10
tensor(1)
10
tensor(1)
10
tensor(1)
10
tensor(7)
10
tensor(11)
10
tensor(10)
10
tensor(1)
10
tensor(1)
10
tensor(1)
8
tensor(20)
1
tensor(1)
10
tensor(11)
8
tensor(1)
8
tensor(1)
10
tensor(5)
8
tensor(9)
8
tensor(3)
8
tensor(1)
8
tensor(1)
10
tensor(1)
10
tensor(1)
8
tensor(7)
20
tensor(9)
8
tensor(1)
10
tensor(1)
10
tensor(7)
10
tensor(1)
4
tensor(7)
10
tensor(4)
10
tensor(7)
10
tensor(2)
10


Other stuff

In [ ]:
dev_text = dev_set['text']
dev_misinfo = dev_set['misinfo']

In [ ]:
counter = 0
for x in dev_text:
  print(x)
  encoding = tokenizer.encode_plus(
        x,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt"
  )
  _, test_prediction = model(encoding["input_ids"], encoding["attention_mask"])
  print(test_prediction)
  print(dev_misinfo[counter])
  counter += 1

Especially with this specific vaccine. It does not contain a living nor a dead virus in it whatsoever. It’s active ingredient is a tiny little mRNA molecule (we naturally have these in our body) that teaches your immune system how to make a specific protein that can fight covid.
tensor([[0.1247, 0.0998, 0.1028, 0.1421, 0.0864, 0.1051, 0.1339, 0.1327, 0.1193,
         0.1457, 0.1169, 0.1116, 0.1270, 0.0835, 0.0969, 0.1149, 0.1023, 0.0969,
         0.1035, 0.1343]])
{'12': 'disagree'}
Fact check: Bell's palsy among COVID-19 trial participants likely unrelated to Pfizer vaccine https://t.co/TGc52w57GB via @Yahoo
tensor([[0.1274, 0.1006, 0.1281, 0.1481, 0.0923, 0.0970, 0.1276, 0.1435, 0.0850,
         0.1635, 0.1157, 0.1278, 0.1046, 0.0985, 0.0910, 0.1121, 0.0945, 0.1001,
         0.0787, 0.1001]])
{'4': 'disagree'}
Can the virus mutate and affect the vaccine’s efficacy?
So far, the virus that causes COVID-19 does not appear to functionally change in an important way.
tensor([[0.1239, 0.10

In [ ]:
_, test_prediction = model(encoding["input_ids"], encoding["attention_mask"])
test_prediction

tensor([[0.1215, 0.1020, 0.1163, 0.1268, 0.0988, 0.1047, 0.1210, 0.1459, 0.1097,
         0.1463, 0.1129, 0.1204, 0.1166, 0.1364, 0.1056, 0.1156, 0.1055, 0.1090,
         0.1027, 0.1266]])

Predicting for Month to Month sets

In [ ]:
pd.read_csv('gdrive/MyDrive/BERT/Datasets/train.csv', index_col=0)

,created_at,id,text,misinfo
0,2020-12-19T04:05:32.000Z,1340146221600526336,The instructions tell the cell to create a pro...,{'5': 'not_relevant'}
1,2020-12-20T08:13:05.000Z,1340570906662322176,@armyveteran13 I said when the virus was first...,{'2': 'agree'}
2,2020-12-20T00:17:11.000Z,1340451144800284672,@sailorrooscout do you have the little image w...,{'20': 'disagree'}
3,2020-12-31T19:17:28.000Z,1344724371865182208,Bombshell Interview With Del Bigtree: COVID Va...,{'2': 'agree'}
4,2021-01-03T12:31:01.000Z,1345709249469546496,"YOLO- berichtgeving: \n\n""COVID Vaccine Caused...",{'4': 'agree'}
...,...,...,...,...
1110,2020-12-20T13:06:19.000Z,1340644700244635648,@MarshaBlackburn Do I still have the freedom t...,{'4': 'agree'}
1111,2020-12-27T23:14:54.000Z,1343334571065176064,they are saying they should get the vaccine al...,{'17': 'not_relevant'}
1112,2020-12-18T19:18:11.000Z,1340013508109414400,Rumor: The COVID-19 vaccine causes infertility...,{'2': 'disagree'}
1113,2020-12-28T00:18:28.000Z,1343350567696027648,"The risk of death or ""long COVID"" if you catch...",{'9': 'disagree'}


In [ ]:
march20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/march20.csv', index_col=0)
april20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/april20.csv', index_col=0)
may20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/may20.csv', index_col=0)
june20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/june20.csv', index_col=0)
july20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/july20.csv', index_col=0)
aug20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/aug20.csv', index_col=0)
sept20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/sept20.csv', index_col=0)
oct20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/oct20.csv', index_col=0)
nov20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/nov20.csv', index_col=0)
dec20 = pd.read_csv('gdrive/MyDrive/BERT/MTM/dec20.csv', index_col=0)
jan21 = pd.read_csv('gdrive/MyDrive/BERT/MTM/jan21.csv', index_col=0)
feb21 = pd.read_csv('gdrive/MyDrive/BERT/MTM/feb21.csv', index_col=0)
march21 = pd.read_csv('gdrive/MyDrive/BERT/MTM/march21.csv', index_col=0)
april21 =pd.read_csv('gdrive/MyDrive/BERT/MTM/april21.csv', index_col=0)
may21 =pd.read_csv('gdrive/MyDrive/BERT/MTM/june20.csv', index_col=0)
june21 =pd.read_csv('gdrive/MyDrive/BERT/MTM/june21.csv', index_col=0)
july21 = pd.read_csv('gdrive/MyDrive/BERT/MTM/july21.csv', index_col=0)
aug21 = pd.read_csv('gdrive/MyDrive/BERT/MTM/august21.csv', index_col=0)
sept21 = pd.read_csv('gdrive/MyDrive/BERT/MTM/sept21.csv', index_col=0)
oct21 = pd.read_csv('gdrive/MyDrive/BERT/MTM/oct21.csv', index_col=0)
nov21 =pd.read_csv('gdrive/MyDrive/BERT/MTM/nov21.csv', index_col=0)
dec21 = pd.read_csv('gdrive/MyDrive/BERT/MTM/dec21.csv', index_col=0)

In [ ]:
def predict_mtm(month, model1):
  text = month['text']
  predictions = []
  for x in text:
    encoding = tokenizer.encode_plus(
        x,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt"
    )
    _, test_prediction = model1(encoding["input_ids"], encoding["attention_mask"])
    predictions.append(test_prediction)
  return predictions


In [ ]:
march20_predictions = predict_mtm(march20, model)

In [ ]:
print(march20_predictions)